# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
file_path_list = []

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    for file in files:
        if '.csv' in file:
            temp_filepath = f'{root}/{file}'
            file_path_list.append(temp_filepath)
        else:
            pass

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length','level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
except "Couldn't connect to local Cassandra instance" as e:
    print(e)

session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS nosql_project 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('nosql_project')
except Exception as e:
    print(e)    

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Create Tables

In [8]:
create_table_sql_template = """CREATE TABLE IF NOT EXISTS {table_name} ({columns}, PRIMARY KEY({primary_key})) {order_by}"""


def create_table(table_name, columns, primary_key, order_by):
    sql = create_table_sql_template.format(
          table_name=table_name
        , columns=columns
        , primary_key=primary_key
        , order_by=order_by
    )
    try:
        session.execute(sql)
        print(f"Created table/query: {table_name}")
    except Exception as e:
        print(e)



create_table(table_name = 'song_info_by_session', columns='artist text, song text, song_length float, session_id int, item_in_session int', primary_key='session_id, item_in_session', order_by='')
create_table(table_name = 'song_and_artist_by_name', columns='artist text, song text, user_first_name text, user_last_name text, user_id int, session_id int, item_in_session int', primary_key='(user_id, session_id), item_in_session', order_by='')
create_table(table_name = 'user_name_by_music_app_history', columns='song text, user_id int, user_first_name text, user_last_name text', primary_key='song, user_id, user_first_name, user_last_name', order_by='')


Created table/query: song_info_by_session
Created table/query: song_and_artist_by_name
Created table/query: user_name_by_music_app_history


In [9]:
## Create Insert Function
def create_insert_values_statement(table_name, columns, number_of_values, value_insert):
    query = f"""INSERT INTO {table_name} ({columns}) VALUES ({number_of_values})"""
    session.execute(query, (value_insert))

In [10]:
#Insert values into artist_songs (query 1)
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        #Insert values into artist_songs (query 1 (song_info_by_session))
        create_insert_values_statement(
                table_name = 'song_info_by_session'
              , columns = '"artist", "song", "song_length", "session_id", "item_in_session"'
              , number_of_values = '%s, %s, %s, %s, %s'
              , value_insert = (line[0], line[9], float(line[5]), int(line[8]), int(line[3]))
        )
        
        #Insert values into artist_songs (query 2 (song_and_artist_by_name))
        create_insert_values_statement(
                table_name = 'song_and_artist_by_name'
              , columns='"artist", "song", "user_first_name", "user_last_name", "user_id", "session_id", "item_in_session"'
              , number_of_values = '%s, %s, %s, %s, %s, %s, %s'
              , value_insert = (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3]))
        )
        
        #Insert values into artist_songs (query 3 (user_name_by_music_app_history))
        create_insert_values_statement(
                table_name = 'user_name_by_music_app_history'
              , columns='"song", "user_id", "user_first_name", "user_last_name"'
              , number_of_values = '%s, %s, %s, %s'
              , value_insert = (line[9],int(line[10]), line[1], line[4])
        )

#### Query 1 (song_info_by_session):  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4


In [11]:
"""
This query returns the artist, song and song length 
for session_id = 338 and item_in_session = 4.
"""

query_song_info_by_session = """select
                  artist
                , song
                , song_length 
            from song_info_by_session 
            where session_id = 338 
            and item_in_session = 4"""

try:
    df1 = pd.DataFrame(list(session.execute(query_song_info_by_session)))
except Exception as e:
    print(e)
    
df1

,artist,song,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


#### Query 2 (song_and_artist_by_name): Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [12]:
"""
This query returns the artist, song and user name (first & last name)
for user_id = 10 and session_id = 182, filtered by itemInSession.
"""

query_song_and_artist_by_name = """select 
                  artist
                , song
                , user_first_name
                , user_last_name
            from song_and_artist_by_name 
            where user_id = 10 
            and session_id = 182 
            order by item_in_session"""

try:
    df2 = pd.DataFrame(list(session.execute(query_song_and_artist_by_name)))
except Exception as e:
    print(e)
    
df2

,artist,song,user_first_name,user_last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### Query 3 (user_name_by_music_app_history): Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [13]:
"""
This query returns the users first and last name 
for users who listened to the song 'All Hands Against my Own'.
"""


query_user_name_by_music_app_history = """select 
                  user_first_name
                , user_last_name 
            from user_name_by_music_app_history 
            where song = 'All Hands Against His Own'
            """

try:
    df3 = pd.DataFrame(list(session.execute(query_user_name_by_music_app_history)))
except Exception as e:
    print(e)
    
df3

,user_first_name,user_last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [14]:
session.execute("""DROP TABLE song_info_by_session""")
session.execute("""DROP TABLE song_and_artist_by_name""")
session.execute("""DROP TABLE user_name_by_music_app_history""")

### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()